In [4]:
import pandas as pd
import folium
import math

## Option 1) If you are using multiple momths of data use the following block

In [5]:
# # list of dates for the downloaded files
# # '201901','201902','201903','201904','201905','201906','201907','201908','201909','201910','201911','201912',
# #             '202001','202002','202003','202004','202005','202006','202007','202008','202009','202010','202011','202012',
# #             '202101','202102','202103','202104','202105','202106', '202107','202108','202109'
# lstDates = ['202207'
#            ]

# lstdf = []
# # a loop to read the monthly csv files 
# for date in lstDates:
#     try:
#         # set the DIRECTORY of the citibike data
#         # read each month's csv
#         pathname = r"DIRECTORY\{}-citibike-tripdata.csv".format(date,date)
#         # add (append) this to a list
#         lstdf.append(pd.read_csv(pathname))
#     except:
#         print('got an error for ',date)
# dfall = pd.concat(lstdf)

## Option 2) If you are using only one month data then set tha pathname here

In [ ]:
pathName = r"Pathname.csv"
dfall = pd.read_csv(pathName)

In [9]:
dfall.sample(2)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
3264188,A1A2D6E9F6A3D2D4,classic_bike,2022-07-25 15:29:32,2022-07-25 15:32:57,E 6 St & Avenue D,5506.14,Avenue D & E 10 St,5575.13,40.722281,-73.976687,40.725066,-73.975158,member
3477610,D2644A463EF9F524,classic_bike,2022-07-14 08:39:45,2022-07-14 08:59:11,E 16 St & 5 Ave,6022.04,E 53 St & Lexington Ave,6617.09,40.737262,-73.992390,40.758281,-73.970694,member


In [10]:
dfall.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3497390 entries, 0 to 3497389
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 346.9+ MB


In [65]:
# convert the columns to datetime object
dfall['started_at'] = pd.to_datetime(dfall['started_at'],format='%Y-%m-%d')
dfall['ended_at'] = pd.to_datetime(dfall['ended_at'],format='%Y-%m-%d')
# calculate duration of each trip
dfall['tripduration'] = dfall['ended_at']-dfall['started_at']


# Convert the timedelta column to numeric values in seconds
dfall.loc[:, 'tripDuration_seconds'] = dfall['tripduration'].apply(lambda x: x.total_seconds())

dfall.loc[:,'tripDuration_minutes'] = dfall['tripDuration_seconds']/60




In [66]:
dfall['started_hour'] = (dfall['started_at'].dt.hour % 24 + 4) // 4
dfall['started_hour'].replace({1: 'Late Night',
                      2: 'Early Morning',
                      3: 'Morning',
                      4: 'Noon',
                      5: 'Evening',
                      6: 'Night'}, inplace=True)

In [67]:
dfall['started_hour'].unique()

array(['Morning', 'Evening', 'Early Morning', 'Night', 'Noon',
       'Late Night'], dtype=object)

In [68]:
dfall['member_casual'].unique()

array(['member', 'casual'], dtype=object)

In [69]:
dfall['rideable_type'].unique()

array(['classic_bike', 'electric_bike', 'docked_bike'], dtype=object)

In [70]:
dfall['start_station_id'] = pd.to_numeric(dfall['start_station_id'], errors='coerce')
dfall = dfall.loc[~dfall['start_station_id'].isna()]
dfall['end_station_id'] = pd.to_numeric(dfall['end_station_id'], errors='coerce')
dfall = dfall.loc[~dfall['end_station_id'].isna()]

In [71]:
dfSt = dfall.groupby(['start_station_id']).agg({'start_lat':['first'],'start_lng':['first']})

In [72]:
dfSt = dfall.groupby(['start_lat','start_lng']).agg({'start_station_id':['first']})

In [73]:
dfEnd = dfall.groupby(['end_station_id']).agg({'end_lat':['first'],'end_lng':['first']})

In [74]:
df =pd.concat([dfSt,dfEnd])

In [75]:
dfSt = dfall.groupby(['start_station_id']).agg({'start_lat':['first'],'start_lng':['first']})
dfSt = dfSt.droplevel(1,axis=1).reset_index()
dfSt.drop_duplicates(subset=['start_lat','start_lng'],keep='first',ignore_index=True,inplace=True)


In [76]:
len(dfSt)

1573

### Mapping stations

In [77]:
# create a dataframe that is consolidated based on start station; get the first lat and long value (all lat and long values should be the same)
dfStarts = (dfall[['start_station_id','start_lat','start_lng']]).groupby(['start_station_id']).agg({'start_lat':['first'],'start_lng':['first']})
# create a dataframe that is consolidated based on end station; get the first lat and long value (all lat and long values should be the same)
dfEnds = (dfall[['end_station_id','end_lat','end_lng']]).groupby(['end_station_id']).agg({'end_lat':['first'],'end_lng':['first']})

In [78]:
# fix the levels
dfStarts = dfStarts.droplevel(1,axis=1).reset_index()
dfEnds = dfEnds.droplevel(1,axis=1).reset_index()

In [79]:
# rename the columns to make them consistent
dfEnds = dfEnds.rename(columns={'end_station_id': 'station_id', 'end_lat': 'latitude','end_lng': 'longitude'})
dfStarts = dfStarts.rename(columns={'start_station_id': 'station_id', 'start_lat': 'latitude','start_lng': 'longitude'})

In [80]:
dfStations = pd.concat([dfStarts,dfEnds])
# convert station_id to a numeric variable



In [81]:
dfStations.sample(2)

,station_id,latitude,longitude
1249,7855.03,40.816960,-73.942296
743,6190.03,40.743534,-74.003676


In [82]:
dfStations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3146 entries, 0 to 1572
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   station_id  3146 non-null   float64
 1   latitude    3146 non-null   float64
 2   longitude   3146 non-null   float64
dtypes: float64(3)
memory usage: 98.3 KB


In [83]:
dfStations.drop_duplicates(subset=None, keep="first", inplace=True)

In [84]:
print ('the total number of stations is: ', len(dfStations))

the total number of stations is:  1907


In [85]:
# create a list of points
locations = dfStations[['latitude', 'longitude','station_id']]
locationlist = locations.values.tolist()
len(locationlist)

1907

In [86]:
# use Folium to map the points
map = folium.Map(location=[dfStations['latitude'].mean(), dfStations['longitude'].mean()], zoom_start=11)
for point in locationlist:
    folium.Circle(
        radius=5,                                                     # the size of circles
        location=point[0:2],                                          # lat and long of the points
        popup='Id: {}'.format(point[2]), # the popup message
        color="black",                                                # color of the circle
        fill=True,
    ).add_to(map)
map

### Station connections

In [109]:
dfStations.to_csv(r"stations.csv")

In [108]:
dfStations.sample()

,station_id,latitude,longitude
1571,8811.01,40.880294,-73.886065


In [87]:
# choose the id of a station
thisStation = 3303.01
# choose the bike type
# ['docked_bike', 'classic_bike']
bikeType = ['docked_bike', 'classic_bike']
# choose member type
#  ['member', 'casual']
memberType = ['member', 'casual']
# choose the startHour
# ['Evening', 'Night', 'Morning', 'Noon', 'Early Morning', 'Late Night']
tripHour = ['Evening', 'Night', 'Morning', 'Noon', 'Early Morning', 'Late Night']

In [88]:
# get the lat and long of the station
thisCoors = (dfStations.loc[(dfStations['station_id']==thisStation)])[['latitude','longitude']]
thisCoors =thisCoors.values.tolist()[0]
thisCoors

[40.649681, -73.96782900000001]

In [89]:
# filter the big table to separate all trips from one start station
dfThis = dfall.loc[(dfall['start_station_id']==thisStation)&
                   (dfall['rideable_type'].isin(bikeType))&
                   (dfall['member_casual'].isin(memberType))&
                   (dfall['started_hour'].isin(tripHour))
                  ]
dfThis.sample(2)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,tripduration,started_hour,tripDuration_seconds,tripDuration_minutes
303204,8BF097F0790BE22A,classic_bike,2022-07-24 12:56:05,2022-07-24 13:21:08,Caton Ave & Argyle Rd,3303.01,Lawrence St & Willoughby St,4596.09,40.649717,-73.967839,40.692362,-73.986317,member,0 days 00:25:03,Noon,1503.0,25.05
439903,D0568E3BA507AD0A,classic_bike,2022-07-17 23:31:24,2022-07-17 23:53:06,Caton Ave & Argyle Rd,3303.01,Parade Pl & Crooke Ave,3376.02,40.649681,-73.967829,40.651847,-73.965248,casual,0 days 00:21:42,Night,1302.0,21.70


In [90]:
len(dfThis)

517

In [91]:
# dfall.groupby(['end_station_id']).agg({'end_lat':['first'],'end_lng':['first']})

In [94]:
dfThis.sample()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,tripduration,started_hour,tripDuration_seconds,tripDuration_minutes
1015535,0426F946D6DD5002,classic_bike,2022-07-17 11:41:23,2022-07-17 12:02:44,Caton Ave & Argyle Rd,3303.01,Douglass St & 4 Ave,4175.14,40.649681,-73.967829,40.679279,-73.98154,member,0 days 00:21:21,Morning,1281.0,21.35


In [95]:
# groupby the filtered dataframe based on the station id of the destinations (carry some variables such as number of trips, average of duration)
dfThisDest = dfThis.groupby(['end_station_id']).agg({'tripDuration_minutes':['count','mean']})
dfThisDest = dfThisDest.droplevel(0,axis=1).reset_index()
dfThisDest = dfThisDest.rename(columns={'end_station_id': 'station_id', 'mean': 'tripDurationMean_Minutes'})
dfThisDest.sample(2)

,station_id,count,tripDurationMean_Minutes
141,4847.03,1,44.216667
2,3007.05,2,8.091667


In [96]:
print('the number of destinations from station id: {} is: '.format(thisStation),len(dfThisDest))

the number of destinations from station id: 3303.01 is:  165


In [97]:
# if we merge/join this with the station tables that has coordinates, then we have locations!

dfThisMergedDest = pd.merge(dfStations,dfThisDest,on='station_id',how='inner')
dfThisMergedDest.sample(2)


,station_id,latitude,longitude,count,tripDurationMean_Minutes
5,3117.05,40.644743,-74.003754,1,38.966667
49,3722.04,40.664946,-73.976307,6,16.705556


In [98]:
# now let's map them!

locations = dfThisMergedDest[['latitude', 'longitude','station_id','count','tripDurationMean_Minutes']]
locationlist = locations.values.tolist()

map2 = folium.Map(location=[dfThisMergedDest['latitude'].mean(), dfThisMergedDest['longitude'].mean()], zoom_start=12)

for point in locationlist:
    folium.Circle(
        radius=math.sqrt(point[3])*15,
        location=point[0:2],
        popup='Id: {}; trip counts: {}; averageDuration {}'.format(point[2],point[3],point[4]),
        color="blue",
        fill=True,
    ).add_to(map2)
    
    
# the location of origin station  
folium.Circle(
        radius=30,
        location=thisCoors,
        popup='your stations',
        color="red",
        fill=True,
    ).add_to(map2)
    
map2

### Let's create a fetch function!

In [104]:

def fetchStation(thisStation,bikeType,memberType,tripHour):
    # get the lat and long of the station
    thisCoors = (dfStations.loc[(dfStations['station_id']==thisStation)])[['latitude','longitude']]
    thisCoors =thisCoors.values.tolist()[0]
    # filter the big table to separate all trips from one start station
    dfThis = dfall.loc[(dfall['start_station_id']==thisStation)&
                   (dfall['rideable_type'].isin(bikeType))&
                   (dfall['member_casual'].isin(memberType))&
                   (dfall['started_hour'].isin(tripHour))
                  ]
    # groupby the filtered dataframe based on the station id of the destinations (carry some variables such as number of trips, average of duration)
    dfThisDest = dfThis.groupby(['end_station_id']).agg({'tripDuration_minutes':['count','mean']})
    dfThisDest = dfThisDest.droplevel(0,axis=1).reset_index()
    dfThisDest = dfThisDest.rename(columns={'end_station_id': 'station_id', 'mean': 'tripDuration_minutes'})
    dfThisMergedDest = pd.merge(dfStations,dfThisDest,on='station_id',how='inner')
    locations = dfThisMergedDest[['latitude', 'longitude','station_id','count','tripDuration_minutes']]
    locationlist = locations.values.tolist()

    map2 = folium.Map(location=[dfThisMergedDest['latitude'].mean(), dfThisMergedDest['longitude'].mean()], zoom_start=12)

    for point in locationlist:
        folium.Circle(
            radius=math.sqrt(point[3])*15,
            location=point[0:2],
            popup='Id: {}; trip counts: {}; averageDuration {}'.format(point[2],point[3],point[4]),
            color="blue",
            fill=True,
        ).add_to(map2)


    # the location of origin station  
    folium.Circle(
            radius=50,
            location=thisCoors,
            popup='your stations',
            color="red",
            fill=True,
        ).add_to(map2)
    
    return map2


In [105]:
# choose the id of a station
thisStation = 3376.02
# choose the bike type
# ['docked_bike', 'classic_bike']
bikeType = ['docked_bike', 'classic_bike']
# choose member type
#  ['member', 'casual']
memberType = ['member', 'casual']
# choose the startHour
# ['Early Morning','Morning','Noon','Evening', 'Night', 'Late Night']
tripHour = ['Early Morning','Morning','Noon','Evening', 'Night', 'Late Night']

In [107]:
fetchStation(thisStation,bikeType,memberType,tripHour)